# Position Monitor

Exposure dashboard for the WolfpackTrend strategy using ObjectStore data.

**Inputs:**
- `{TEAM_ID}/positions.csv` - All positions daily
- `{TEAM_ID}/daily_snapshots.csv` - Daily NAV, exposure metrics

**Features:**
- Top weights on a given date
- Net/gross/long/short exposure over time
- Position count trends

**Usage:**
- Run the backtest to populate ObjectStore, then run all cells.
- Set `target_date` to inspect a specific date.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from datetime import datetime
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook
from config import TEAM_ID

qb = QuantBook()
print("QuantBook initialized")


def read_csv_from_store(key):
    """Read CSV from ObjectStore with error handling."""
    try:
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f"Empty ObjectStore key: {key}")
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f"Error reading {key}: {e}")
        return None

## Load Data

In [ ]:
df_positions = read_csv_from_store(f"{TEAM_ID}/positions.csv")
df_snapshots = read_csv_from_store(f"{TEAM_ID}/daily_snapshots.csv")

if df_positions is not None:
    df_positions["date"] = pd.to_datetime(df_positions["date"])

if df_snapshots is not None:
    df_snapshots["date"] = pd.to_datetime(df_snapshots["date"])

print(
    "Loaded:",
    f"positions={len(df_positions) if df_positions is not None else 0}",
    f"snapshots={len(df_snapshots) if df_snapshots is not None else 0}",
)

## Configuration

In [ ]:
# Set target_date to inspect a specific date, or None for latest
target_date = None  # e.g., "2023-06-30"

if df_positions is None:
    raise ValueError("positions.csv is required for this notebook")

if target_date:
    target_date = pd.to_datetime(target_date)
else:
    target_date = df_positions["date"].max()

print(f"Target date: {target_date.strftime('%Y-%m-%d')}")

## Portfolio Snapshot

In [ ]:
if df_snapshots is not None and len(df_snapshots) > 0:
    latest_snapshot = df_snapshots[df_snapshots["date"] == target_date]
    if len(latest_snapshot) == 0:
        latest_snapshot = df_snapshots.sort_values("date").tail(1)
        print("No snapshot for target date; using latest available snapshot.")
    snapshot = latest_snapshot.iloc[0]

    # Display key metrics
    snapshot_cols = [
        "date", "nav", "cash",
        "gross_exposure", "net_exposure", 
        "long_exposure", "short_exposure",
        "daily_pnl", "cumulative_pnl",
        "num_positions", "estimated_vol"
    ]
    snapshot_cols = [c for c in snapshot_cols if c in snapshot.index]
    
    print("\nPortfolio Snapshot:")
    print("=" * 60)
    display(snapshot[snapshot_cols].to_frame("Value"))
else:
    print("No daily snapshots data available")

## Positions on Target Date

In [ ]:
df_day = df_positions[df_positions["date"] == target_date].copy()
print(f"Position records for {target_date.strftime('%Y-%m-%d')}: {len(df_day)}")

if len(df_day) == 0:
    print("No positions found for target date.")
else:
    # Show all positions sorted by absolute weight
    if 'weight' in df_day.columns:
        df_day['abs_weight'] = df_day['weight'].abs()
        df_day = df_day.sort_values('abs_weight', ascending=False)
        
        display_cols = ['symbol', 'weight', 'quantity', 'price']
        display_cols = [c for c in display_cols if c in df_day.columns]
        if display_cols:
            display(df_day[display_cols])
    else:
        display(df_day)

## Top Weights on Target Date

In [ ]:
if len(df_day) > 0 and 'weight' in df_day.columns:
    # Separate long and short positions
    long_positions = df_day[df_day['weight'] > 0].copy()
    short_positions = df_day[df_day['weight'] < 0].copy()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Top 10 long positions
    if len(long_positions) > 0:
        top_long = long_positions.nlargest(10, 'weight')
        symbol_col = 'symbol' if 'symbol' in top_long.columns else top_long.columns[0]
        axes[0].barh(top_long[symbol_col], top_long['weight'] * 100, color='green', alpha=0.7)
        axes[0].set_title(f'Top 10 Long Positions ({target_date.strftime("%Y-%m-%d")})', fontsize=12)
        axes[0].set_xlabel('Weight (%)')
        axes[0].invert_yaxis()
        axes[0].grid(True, alpha=0.3, axis='x')
    else:
        axes[0].text(0.5, 0.5, 'No long positions', ha='center', va='center')
        axes[0].set_title('Top 10 Long Positions')
    
    # Top 10 short positions (by absolute weight)
    if len(short_positions) > 0:
        top_short = short_positions.nsmallest(10, 'weight')
        symbol_col = 'symbol' if 'symbol' in top_short.columns else top_short.columns[0]
        axes[1].barh(top_short[symbol_col], top_short['weight'] * 100, color='red', alpha=0.7)
        axes[1].set_title(f'Top 10 Short Positions ({target_date.strftime("%Y-%m-%d")})', fontsize=12)
        axes[1].set_xlabel('Weight (%)')
        axes[1].invert_yaxis()
        axes[1].grid(True, alpha=0.3, axis='x')
    else:
        axes[1].text(0.5, 0.5, 'No short positions', ha='center', va='center')
        axes[1].set_title('Top 10 Short Positions')
    
    plt.tight_layout()
    plt.show()
else:
    print("No weight data available for position analysis")

## Exposure Over Time

In [ ]:
if df_snapshots is not None and len(df_snapshots) > 0:
    exposure_cols = ['gross_exposure', 'net_exposure', 'long_exposure', 'short_exposure']
    exposure_cols = [c for c in exposure_cols if c in df_snapshots.columns]
    
    if exposure_cols:
        fig, ax = plt.subplots(figsize=(14, 6))
        
        for col in exposure_cols:
            ax.plot(df_snapshots['date'], df_snapshots[col] * 100, linewidth=2, label=col)
        
        ax.set_title('Portfolio Exposure Over Time', fontsize=14, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel('Exposure (% of NAV)')
        ax.legend(loc='upper left')
        ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
        
        # Exposure statistics
        print("\nExposure Statistics:")
        print("=" * 60)
        print(df_snapshots[exposure_cols].describe())
    else:
        print("No exposure columns found in daily snapshots")
else:
    print("No daily snapshots data available")

## Position Count Over Time

In [ ]:
if df_positions is not None and len(df_positions) > 0:
    # Count positions per day
    daily_counts = df_positions.groupby('date').size().reset_index(name='num_positions')
    
    # Also separate by direction if weight column exists
    if 'weight' in df_positions.columns:
        long_counts = df_positions[df_positions['weight'] > 0].groupby('date').size().reset_index(name='long_positions')
        short_counts = df_positions[df_positions['weight'] < 0].groupby('date').size().reset_index(name='short_positions')
        daily_counts = daily_counts.merge(long_counts, on='date', how='left')
        daily_counts = daily_counts.merge(short_counts, on='date', how='left')
        daily_counts = daily_counts.fillna(0)
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    ax.plot(daily_counts['date'], daily_counts['num_positions'], linewidth=2, label='Total', color='blue')
    
    if 'long_positions' in daily_counts.columns:
        ax.plot(daily_counts['date'], daily_counts['long_positions'], linewidth=2, label='Long', color='green', alpha=0.7)
    if 'short_positions' in daily_counts.columns:
        ax.plot(daily_counts['date'], daily_counts['short_positions'], linewidth=2, label='Short', color='red', alpha=0.7)
    
    ax.set_title('Active Position Count Over Time', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Number of Positions')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No positions data available")

## Weight Distribution by Symbol Over Time

In [ ]:
if df_positions is not None and len(df_positions) > 0 and 'weight' in df_positions.columns:
    # Get top symbols by average absolute weight
    symbol_col = 'symbol' if 'symbol' in df_positions.columns else df_positions.columns[1]
    avg_weights = df_positions.groupby(symbol_col)['weight'].apply(lambda x: x.abs().mean()).sort_values(ascending=False)
    top_symbols = avg_weights.head(10).index.tolist()
    
    # Pivot for time series
    pivot = df_positions[df_positions[symbol_col].isin(top_symbols)].pivot_table(
        index='date', columns=symbol_col, values='weight', aggfunc='sum', fill_value=0
    ).sort_index()
    
    if len(pivot.columns) > 0:
        fig, ax = plt.subplots(figsize=(14, 6))
        
        (pivot * 100).plot(ax=ax, linewidth=2, alpha=0.8)
        
        ax.set_title('Weight Over Time (Top 10 Symbols by Avg Absolute Weight)', fontsize=14, fontweight='bold')
        ax.set_xlabel('Date')
        ax.set_ylabel('Weight (%)')
        ax.legend(loc='upper left', ncol=2)
        ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
else:
    print("No weight data available for symbol analysis")

## Volatility Target Tracking

In [ ]:
if df_snapshots is not None and 'estimated_vol' in df_snapshots.columns:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Plot estimated volatility
    ax.plot(df_snapshots['date'], df_snapshots['estimated_vol'] * 100, linewidth=2, label='Estimated Vol', color='steelblue')
    
    # Add target volatility line (10% from strategy parameters)
    target_vol = 10.0
    ax.axhline(y=target_vol, color='red', linestyle='--', linewidth=2, label=f'Target Vol ({target_vol}%)')
    
    ax.set_title('Estimated Portfolio Volatility vs Target', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Annualized Volatility (%)')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("\nVolatility Statistics:")
    print("=" * 60)
    print(f"Mean estimated vol: {df_snapshots['estimated_vol'].mean() * 100:.2f}%")
    print(f"Std of estimated vol: {df_snapshots['estimated_vol'].std() * 100:.2f}%")
    print(f"Min estimated vol: {df_snapshots['estimated_vol'].min() * 100:.2f}%")
    print(f"Max estimated vol: {df_snapshots['estimated_vol'].max() * 100:.2f}%")
else:
    print("No estimated_vol column found in daily snapshots")

## Summary by Symbol

In [ ]:
if df_positions is not None and len(df_positions) > 0:
    symbol_col = 'symbol' if 'symbol' in df_positions.columns else df_positions.columns[1]
    
    agg_dict = {'date': 'count'}
    if 'weight' in df_positions.columns:
        agg_dict['weight'] = ['mean', 'std', 'min', 'max']
    
    summary = df_positions.groupby(symbol_col).agg(agg_dict)
    summary.columns = ['_'.join(col).strip('_') for col in summary.columns.values]
    summary = summary.rename(columns={'date_count': 'days_held'})
    summary = summary.sort_values('days_held', ascending=False)
    
    print("\nPosition Summary by Symbol:")
    print("=" * 80)
    display(summary.head(20))
else:
    print("No positions data available")